## [분야2] 서비스 품질 보장을 위한 플로우 특성 실시간 식별 베이스라인 코드
#### 주의: 반드시 본 파일을 이용하여 제출을 수행해야 하며 파일의 이름은 task.ipynb로 유지되어야 합니다.

### 추론 실행 환경
1. python 3.10 환경
2. torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
    - 최신 버전입니다.

코드는 크게 3가지 파트로 구성되며, 해당 파트의 특성을 지켜서 내용을 편집하시면 되겠습니다.
1. 제출용 aifactory 라이브러리 및 추가 필요 라이브러리 설치
    - 채점 및 제출을 위한 aifactory 라이브러리를 설치하는 셀입니다. 이 부분은 수정하지 않고 그대로 실행합니다.
    - 필요 라이브러리를 직접 설치합니다.
2. 추론용 코드 작성
    - 모델 로드, 데이터 전처리, 예측 등 실제 추론을 수행하는 모든 코드를 이 영역에 작성합니다.
3. aif.submit() 함수를 호출하여 최종 결과를 제출하는 파트입니다.
    - 마이 페이지에서 발급받은 key 값을 함수의 인자로 정확히 입력해야 합니다.

※ 가능하면 제출시에는 포함되어 있는 train data를 폴더에서 제외하고 제출하시는 편이 좋습니다.
    - 파일 크기 감소 → 업로드 시간 감소 → 전체 추론 수행 시간 감소

### 1. 제출용 aifactory 라이브러리 설치
#### 결과 전송에 필요하므로 아래와 같이 aifactory 라이브러리가 반드시 최신버전으로 설치될 수 있게끔 합니다

In [1]:
!pip install -U aifactory

  Using cached ipython-8.12.3-py3-none-any.whl.metadata (5.7 kB)
Using cached ipython-8.12.3-py3-none-any.whl (798 kB)
  Attempting uninstall: IPython
    Found existing installation: ipython 8.28.0
    Uninstalling ipython-8.28.0:
      Successfully uninstalled ipython-8.28.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 6.0.7 requires ipython!=8.17.1,<9.0.0,>=8.13.0; python_version > "3.8", but you have ipython 8.12.3 which is incompatible.
spyder-kernels 3.0.5 requires ipython!=8.17.1,<9,>=8.13.0; python_version > "3.8", but you have ipython 8.12.3 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:
!pip install -U pytorch-lightning==2.0.9
!pip install -U scikit-learn==1.3.0
!pip install -U numpy==1.23.5
!pip install pexpect
!pip install decorator
!pip install certifi

### 추론 환경의 기본 경로 구조
#### 제출 시 주의사항

1. 테스트 데이터 경로: /aif/data/test_packet_data.pkl
   - 채점에 사용될 테스트 입력 데이터는 이 디렉토리 안에 test_packet_data.pkl이라는 이름으로 고정되어 제공됩니다.
2. 모델 및 자원 경로: 예시 : ./model/
   - 추론 스크립트가 실행되는 위치를 기준으로, 제출된 모델 관련 파일들이 위치하는 상대 경로입니다.
   - 학습된 모델 가중치(.pt, .ckpt, .pth 등)
3. ※ 반드시 입력 패킷 길이 변수명을 반드시 "input_packet_length"로 지정해야 합니다.
   - 해당 변수명이 누락되거나 다를 경우 "input_packet_length 변수를 정상적으로 정의해야 합니다!"라는 오류가 발생합니다.
4. 제출 파일은 submission.pkl로 저장
5. argparse 사용시 args, _ = parser.parse_known_args()로 인자 지정
   args = parser.parse_args()는 jupyter에서 오류가 발생합니다!!!
6. return할 결과물과 양식에 유의합니다.

In [ ]:
import os
import numpy as np
import pandas as pd
import joblib
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import json

# --- 0. 설정 및 초기화 ---
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 1. 테스트 데이터셋 클래스 정의 ---
class FlowPacketTestDataset(Dataset):
    def __init__(self, packet_data_list, seq_len, features):
        self.packet_data_list = packet_data_list
        self.seq_len = seq_len
        self.features = features
        
    def __len__(self):
        return len(self.packet_data_list)
    
    def __getitem__(self, idx):
        packets_df = self.packet_data_list[idx]
        packets_df = packets_df[self.features]
        x = packets_df.head(self.seq_len)
        
        # 패딩 처리 (안전장치)
        x_values = x.values.astype(np.float32)
        if len(x_values) < self.seq_len:
            pad_len = self.seq_len - len(x_values)
            padding = np.zeros((pad_len, x_values.shape[1]), dtype=np.float32)
            x_values = np.vstack([x_values, padding])
            
        x_tensor = torch.FloatTensor(x_values)
        return x_tensor

# --- 2. 추론 전용 모델 클래스 정의 (nn.Module 사용) ---
class InferenceModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True
        )
        self.head_duration = nn.Linear(hidden_size, num_classes)
        self.head_volume = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]
        logits_duration = self.head_duration(last_output)
        logits_volume = self.head_volume(last_output)
        return logits_duration, logits_volume

# --- 3. 저장된 모델과 메타데이터 로드 함수 ---
def load_model_components(model_name, model_path):
    meta_path = os.path.join(model_path, f'{model_name}_meta.json')
    model_weights_path = os.path.join(model_path, f'{model_name}.pth')
    
    print(f"메타데이터 로딩: {meta_path}")
    with open(meta_path, 'r') as f:
        metadata = json.load(f)

    # InferenceModel 클래스 사용
    model = InferenceModel(
        input_size=metadata['input_size'],
        hidden_size=metadata['hidden_size'],
        num_classes=metadata['num_classes']
    )
    
    model.load_state_dict(torch.load(model_weights_path, map_location=device))
    model.to(device)
    model.eval()
    return model, metadata

# --- 4. 데이터 로드 및 모델 컴포넌트 준비 ---
test_packet_data = joblib.load("/aif/data/test_packet_data.pkl")
MODEL_DIR = "./model" 
MODEL_NAME = "traffic_multitask_lstm"
model, metadata = load_model_components(model_name=MODEL_NAME, model_path=MODEL_DIR)

# --- 5. 테스트 데이터셋 및 데이터로더 생성 ---
test_dataset = FlowPacketTestDataset(
    packet_data_list=test_packet_data,
    seq_len=metadata['seq_len'],
    features=metadata['packet_features']
)
BATCH_SIZE = 256 
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=1
)

# --- 6. 추론 실행 ---
pred_duration = []
pred_volume = []
with torch.no_grad():
    for x_batch in tqdm(test_loader, desc="[base_line]"):
        x_batch = x_batch.to(device)
        logits_duration, logits_volume = model(x_batch)
        preds_d = torch.argmax(logits_duration, dim=1)
        preds_v = torch.argmax(logits_volume, dim=1)
        pred_duration.append(preds_d.cpu().numpy())
        pred_volume.append(preds_v.cpu().numpy())

# --- 7. 결과 취합 ---
y_pred_duration = np.concatenate(pred_duration, axis=0)
y_pred_volume = np.concatenate(pred_volume, axis=0)
# 예측 결과를 딕셔너리로 묶음
submission_dict = {
    'duration_class': y_pred_duration,
    'volume_class': y_pred_volume
}

# 딕셔너리를 Pandas DataFrame으로 변환
submission_df = pd.DataFrame(submission_dict)

input_packet_length = 3
# DataFrame을 pkl 파일로 저장
submission_path = "submission.pkl"
joblib.dump(submission_df, submission_path)


### 4. 
#### ※ task별, 참가자별로 key가 다릅니다. 잘못 입력하지 않도록 유의바랍니다.
- key는 각팀 팀장분에게 메일로 할당된 계정을 이용하여 task별로 확인하실 수 있습니다.

In [ ]:
import aifactory.score as aif
import time
t = time.time()

#-----------------------------------------------------#
aif.submit(model_name="My_model",
           key="My_key"
           )
#-----------------------------------------------------#
print(time.time() - t)

file : task.py
python
제출 완료
1.7257747650146484
